# Load Modules and File Paths

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import math
from os.path import abspath
import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.axes_grid1 import make_axes_locatable
matplotlib.rcParams['text.latex.unicode']=False
import datetime
from operator import add
import pickle
from sklearn.impute import SimpleImputer as Imputer
from osgeo import gdal
import xarray as xr
from affine import Affine
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import glob
from pathlib import Path
# import richdem as rd

imp = Imputer(strategy='mean')

# --------------------- #
# Lets Rips Some Treads #
# --------------------- #

from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()


if abspath('../../utils/') not in sys.path:
    sys.path.append(abspath('../../utils/'))
from utils import GetExtent

/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:10: MatplotlibDeprecationWarning: 
The text.latex.unicode rcparam was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
# SITENAME = 'tuolumne'
# name = 'snowdepth'
# formato = 'netcdf'
# YEAR = '2015'
# path_to_data = f'/srv/shared/deep_stac/data/{name}/{formato}/{YEAR}'
# path_to_topo = '/srv/shared/deep_stac/data/topo.nc'
# files = os.listdir(path_to_data)
# files = np.sort([f for f in files if 'int' in f])
# fname_list = [os.path.join(path_to_data, f) for f in np.sort(files)]
# dates = [d.split('_')[0] for d in files]
# dates = np.sort(dates).tolist()
# dates = [pd.to_datetime(d.split('_')[0]).date() for d in files]

In [2]:
import glob
path = '/srv/shared/deep_stac/data/snowdepth/netcdf/20*/*int.nc'
flist = glob.glob(path, recursive=True)
flist

['/srv/shared/deep_stac/data/snowdepth/netcdf/2015/20150403_SUPERsnow_depth_3m_int.nc',
 '/srv/shared/deep_stac/data/snowdepth/netcdf/2015/20150528_SUPERsnow_depth_3m_int.nc',
 '/srv/shared/deep_stac/data/snowdepth/netcdf/2015/20150218_SUPERsnow_depth_3m_int.nc',
 '/srv/shared/deep_stac/data/snowdepth/netcdf/2015/20150306_SUPERsnow_depth_3m_int.nc',
 '/srv/shared/deep_stac/data/snowdepth/netcdf/2015/20150427_SUPERsnow_depth_3m_int.nc',
 '/srv/shared/deep_stac/data/snowdepth/netcdf/2015/20150409_SUPERsnow_depth_3m_int.nc',
 '/srv/shared/deep_stac/data/snowdepth/netcdf/2015/20150415_SUPERsnow_depth_3m_int.nc',
 '/srv/shared/deep_stac/data/snowdepth/netcdf/2015/20150325_SUPERsnow_depth_3m_int.nc',
 '/srv/shared/deep_stac/data/snowdepth/netcdf/2015/20150608_SUPERsnow_depth_3m_int.nc',
 '/srv/shared/deep_stac/data/snowdepth/netcdf/2015/20150501_SUPERsnow_depth_3m_int.nc',
 '/srv/shared/deep_stac/data/snowdepth/netcdf/2017/20170401_SUPERsnow_depth_3m_int.nc',
 '/srv/shared/deep_stac/data/sno

### Read in Lidar and Topo Data

In [4]:
# --------------- #
# Read Lidar Data #
# --------------- #
ds = xr.open_mfdataset(flist, concat_dim='flight_date', chunks={'x':1000, 'y':1000})
ds.close()
ds

# dtopo = xr.open_dataset(path_to_topo, chunks={'x':1000, 'y':1000})
# dtopo.close()

<xarray.Dataset>
Dimensions:              (flight_date: 19, x: 17569, y: 17002)
Coordinates:
  * x                    (x) float64 2.54e+05 2.54e+05 ... 3.067e+05 3.067e+05
  * y                    (y) float64 4.179e+06 4.179e+06 ... 4.23e+06 4.23e+06
Dimensions without coordinates: flight_date
Data variables:
    transverse_mercator  (flight_date) |S1 b'' b'' b'' b'' ... b'' b'' b'' b''
    Band1                (flight_date, y, x) float32 dask.array<shape=(19, 17002, 17569), chunksize=(1, 1000, 1000)>
Attributes:
    GDAL_AREA_OR_POINT:  Area
    Conventions:         CF-1.5
    GDAL:                GDAL 2.4.1, released 2019/03/15
    history:             Wed Sep 11 15:41:18 2019: GDAL CreateCopy( /srv/shar...

### Sub select features to include to the dem. 

In [ ]:
# add_to_lidar = ['dem','veg_height','mask']
# dtopo = dtopo[add_to_lidar]

In [ ]:
# -------------------- #
# Remove This features #
# -------------------- #

ds = ds.drop('transverse_mercator')

dtopo = dtopo.expand_dims('flight_date')

### Print Data Details

In [ ]:
print('TOPO: \n\n',dtopo, '\n\n\nLidar:\n\n', ds)

In [ ]:
ds['date'] = (['date'], dates)
ds = ds.assign_coords(date=ds.date)

In [ ]:
dates

In [ ]:
datos = {}

In [ ]:
datos['snowdepth'] = np.divide(ds.Band1[0,:,:].values,100)
datos['vegh'] = dtopo.veg_height.values.squeeze(axis=0)
datos['dem'] = dtopo.dem.values.squeeze(axis=0)

In [ ]:
print(type(datos['dem']))
print(datos['dem'].shape)

## Generate Topographic Features from DEM

In [ ]:
# datos['dem'] = rd.rdarray(datos['dem'], no_data=-9999)

# datos['slope'] = rd.TerrainAttribute(
#     rd.rdarray(datos['dem'], no_data=-9999),
#     attrib='slope_riserun'
# )
# datos['aspect'] = rd.TerrainAttribute(
#     rd.rdarray(datos['dem'], no_data=-9999), 
#     attrib='aspect')

# datos['curvature'] = rd.TerrainAttribute(
#     rd.rdarray(datos['dem'], no_data=-9999), 
#     attrib='profile_curvature'
# )

In [ ]:
### Visualize Features

In [ ]:
# feat_to_viz = 'aspect'

# plt.figure(figsize=(10,15))
# ax = plt.subplot(1,1,1)
# im = plt.imshow(
#     np.flipud(datos['feat_to_viz']), 
#     origin='lower', 
#     cmap='hsv'
# #     vmin=0, vmax=360
# )
# # create an axes on the right side of ax. The width of cax will be 5%
# # of ax and the padding between cax and ax will be fixed at 0.05 inch.
# divider = make_axes_locatable(ax)
# cax = divider.append_axes("right", size="5%", pad=0.05)

# plt.colorbar(im, cax=cax)

In [ ]:
# boo = DS['Band1'][4,10000:14000,10000:14000]

In [ ]:
# boo.plot()

In [ ]:
# import sys
# print(boo.nbytes/1000000)